In [3]:
import os
from google.cloud import bigquery
import pandas as pd

In [7]:
credentials = os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="ETL-setup.json"

In [9]:
#load data from local machine to bigquery - staging area
def load_csv_to_bigquery(csv_path, project_id, table_name):
    dataset_name = 'staging'
    # Create a BigQuery client using your service account key file
    #credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(project=project_id)

    # Read the CSV file into a Pandas dataframe
    df = pd.read_csv(csv_path)

    # Create the BigQuery dataset if it doesn't exist
    dataset_ref = client.dataset(dataset_name)
    try:
        client.get_dataset(dataset_ref)
        print("Dataset {} already exists".format(dataset_name))
    except:
        print("Creating dataset {}".format(dataset_name))
        dataset = bigquery.Dataset(dataset_ref)
        client.create_dataset(dataset)

    # Set the destination table for the data
    table_ref = dataset_ref.table(table_name)

    # Define the schema of the table
    schema = []
    for column in df.columns:
        schema.append(bigquery.SchemaField(column, 'STRING'))

    # Create the table in BigQuery
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)

    # Load the data into the table
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.skip_leading_rows = 1
    job_config.autodetect = False # Set to True to automatically detect schema, False to use schema defined above
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()

    print("Data uploaded to BigQuery successfully.")

In [13]:
load_csv_to_bigquery("C:\\Users\\UX501VW\\Desktop\\BigQuery-Data-Modeling\\Dastaset\\demographic.csv", "snappy-nomad-382716", "demographic")

Creating dataset staging
Data uploaded to BigQuery successfully.


In [14]:
load_csv_to_bigquery("C:\\Users\\UX501VW\\Desktop\\BigQuery-Data-Modeling\\Dastaset\\referral.csv", "snappy-nomad-382716", "referal")
load_csv_to_bigquery("C:\\Users\\UX501VW\\Desktop\\BigQuery-Data-Modeling\\Dastaset\\transactions.csv", "snappy-nomad-382716", "transactions")

Dataset staging already exists
Data uploaded to BigQuery successfully.
Dataset staging already exists
Data uploaded to BigQuery successfully.
